In [2]:
import pandas as pd
import difflib

In [6]:
# 导入英文单词
data = pd.read_csv("english.csv", index_col=0)
data.head(2)

,word,phonetic
0,"a, an",[ə；æn]
1,abandon,[əˈbændən]


In [7]:
# Clean up.
data['phonetic'] = data['phonetic'].str.strip().str.lstrip('[').str.rstrip(']')
data['phonetic'] = data['phonetic'].apply(lambda x: x[:x.find('；')])
data['word'] = data['word'].apply(lambda x: x[:x.find(', ')])

data.head(2)

,word,phonetic
0,a,ə
1,abando,əˈbændə


In [8]:
# 单词筛选
data.drop_duplicates('word', inplace=True)
data.drop_duplicates('phonetic', inplace=True)
data.reset_index(drop=True, inplace=True)
print(f"{len(data)} words remain.")

2480 words remain.


In [9]:
chinese = u"有备而来"
import dragonmapper.hanzi
chinese_in_ipa = dragonmapper.hanzi.to_ipa(chinese)
print(chinese_in_ipa)

yoʊ˧˩˧ peɪ˥˩ ɑɻ˧˥ laɪ˧˥


In [10]:
chars_in_ipa = chinese_in_ipa.split(' ')
chars_in_ipa

['yoʊ˧˩˧', 'peɪ˥˩', 'ɑɻ˧˥', 'laɪ˧˥']

In [11]:
remove_tone_symbols = lambda char_in_ipa: char_in_ipa.rstrip('˥').rstrip('˧˥').rstrip('˧˩˧').rstrip('˥˩')
chars_in_ipa = list(map(remove_tone_symbols, chars_in_ipa))
chars_in_ipa

['yoʊ', 'peɪ', 'ɑɻ', 'laɪ']

In [12]:
str1 = ''.join(chars_in_ipa[1:3])
data = data.assign(
    diff_sim = data['phonetic'].apply(lambda str2: difflib.SequenceMatcher(None, str1,str2).ratio())
)

In [13]:
print(f'Top 5 most similar words to {str1} by `difflib.SequenceMatcher`:')
data.sort_values('diff_sim', ascending=False).head(5)

Top 5 most similar words to peɪɑɻ by `difflib.SequenceMatcher`:


,word,phonetic,diff_sim
1563,pac,peɪ,0.750000
1635,plac,pleɪ,0.666667
1687,prais,preɪ,0.666667
1567,pain,peɪn,0.666667
2078,spac,speɪ,0.666667
